In [ ]:
import pandas as pd 
import numpy as np 
from load_data import load_and_process_data, create_nutrient_constraints, load_all_menus, load_sample_file
from evaluation_function import calculate_harmony_matrix, get_top_n_harmony_pairs
from utils import diet_to_dataframe, count_menu_changes
import matplotlib.pyplot as plt
import seaborn as sns

from performance_metrics import PerformanceEvaluator
from nsga2_optimizer import NSGA2Optimizer
from nsga3_optimizer import NSGA3Optimizer
from spea2_optimizer import SPEA2Optimizer
from emoea_optimizer import EpsilonMOEAOptimizer

In [ ]:
# 데이터 로딩
def load_data(diet_db_path, menu_db_path, ingre_db_path):    
    diet_db = load_and_process_data(diet_db_path, menu_db_path, ingre_db_path)
    nutrient_constraints = create_nutrient_constraints()
    harmony_matrix, menus, menu_counts, _ = calculate_harmony_matrix(diet_db)
    all_menus = load_all_menus(menu_db_path, ingre_db_path)
    
    return diet_db, nutrient_constraints, harmony_matrix, menus, menu_counts, all_menus

def compare_optimizers(initial_diet_path, diet_db, menu_db_path, ingre_db_path,
                    all_menus, nutrient_constraints, harmony_matrix,
                    generations: int = 100):
    weekly_diet = load_and_process_data(diet_db_path=initial_diet_path,
                                      menu_db_path=menu_db_path,
                                      ingre_db_path=ingre_db_path)
    
    # 최적화 알고리즘 초기화
    optimizers = {
        'NSGA-II': NSGA2Optimizer(all_menus, nutrient_constraints, harmony_matrix),
        'NSGA-III': NSGA3Optimizer(all_menus, nutrient_constraints, harmony_matrix),
        'SPEA2': SPEA2Optimizer(all_menus, nutrient_constraints, harmony_matrix),
        'ε-MOEA': EpsilonMOEAOptimizer(all_menus, nutrient_constraints, harmony_matrix)
    }

    # 초기 식단 분석
    initial_fitness = optimizers['NSGA-III'].fitness(diet_db, weekly_diet)
    
    # 각 알고리즘별 최적화 수행
    all_results = {}
    for name, optimizer in optimizers.items():
        print(f"\n=== {name} 알고리즘 최적화 진행 중... ===")
        pareto_front = optimizer.optimize(diet_db, weekly_diet, generations)
        
        # 개선된 식단 선별
        improved_diets = []
        for optimized_diet in pareto_front:
            optimized_fitness = optimizer.fitness(diet_db, optimized_diet)
            improvements = calculate_improvements(initial_fitness, optimized_fitness)
            if sum(1 for imp in improvements if imp > 0) >= 3:
                improved_diets.append((optimized_diet, optimized_fitness, improvements))
        
        improved_diets = improved_diets[:5]
        all_results[name] = improved_diets
        
        # 결과 출력
        print_optimization_results(name, improved_diets, weekly_diet, nutrient_constraints)
    
    # 알고리즘 성능 비교 (10회 반복 실행)
    print("\n" + "="*60)
    print("알고리즘 성능 비교 (10회 반복 실행 시작)")
    print("="*60)
    
    evaluator = PerformanceEvaluator(diet_db, weekly_diet, optimizers)
    performance_results = {}
    
    # 각 알고리즘을 10회씩 실행하여 성능 측정
    for name in optimizers.keys():
        performance_results[name] = evaluator.run_single_optimizer(
            optimizer_name=name,
            generations=generations,
            num_runs=10
        )
    
    # 결과 저장
    evaluator.save_combined_results_to_excel(
        performance_results,
        filename='optimization_comparison_results.xlsx'
    )
    
    print("\n" + "="*60)
    print("성능 비교 완료! 결과가 'optimization_comparison_results.xlsx'에 저장되었습니다.")
    print("="*60)
    
    return all_results, performance_results

def print_optimization_results(algorithm_name, improved_diets, initial_diet, nutrient_constraints):
    if improved_diets:
        print(f"\n=== {algorithm_name}: 개선된 식단 제안 ===")
        for i, (optimized_diet, optimized_fitness, improvements) in enumerate(improved_diets, 1):
            days = len(optimized_diet.meals) // 3
            print(f"\n[제안 식단 {i}]")
            #display(diet_to_dataframe(optimized_diet, f"{algorithm_name} - Optimized Diet {i}"))
            
            #print(f"\n[제안 식단 {i} 일일 평균 영양성분]")
            for nutrient in nutrient_constraints.min_values.keys():
                total = sum(sum(menu.nutrients[nutrient] for menu in meal.menus) for meal in optimized_diet.meals)
                daily_avg = total / days
                min_val = nutrient_constraints.min_values[nutrient]
                max_val = nutrient_constraints.max_values[nutrient]
                #print(f"{nutrient}: {daily_avg:.1f} (권장범위: {min_val}-{max_val})")
            
            optimized_cost = sum(sum(ingredient.price for menu in meal.menus for ingredient in menu.ingredients) for meal in optimized_diet.meals)
            #print(f"총 식재료 비용: {optimized_cost:,.0f}원")

            print("\n점수 개선율:")
            print(f"영양: {optimized_fitness[0]:.2f} ({improvements[0]:.2f}%)")
            print(f"비용: {optimized_fitness[1]:.2f} ({improvements[1]:.2f}%)")
            print(f"조화: {optimized_fitness[2]:.2f} ({improvements[2]:.2f}%)")
            print(f"다양성: {optimized_fitness[3]:.2f} ({improvements[3]:.2f}%)")
            
            menu_changes = count_menu_changes(initial_diet, optimized_diet)
            #print("\n카테고리별 메뉴 변경 비율:")
            for category, counts in menu_changes.items():
                percentage = (counts['changed'] / counts['total']) * 100 if counts['total'] > 0 else 0
                #print(f"{category}: {counts['changed']}/{counts['total']} ({percentage:.2f}%)")
    else:
        print(f"\n{algorithm_name}: 3가지 이상 개선된 식단을 찾지 못했습니다.")

def calculate_improvements(initial_fitness, optimized_fitness):
    improvements = []
    for init, opt in zip(initial_fitness, optimized_fitness):
        if init != 0:
            imp = (opt - init) / abs(init) * 100
        else:
            imp = float('inf') if opt > 0 else (0 if opt == 0 else float('-inf'))
        improvements.append(imp)
    return improvements

In [ ]:
# 데이터 로드
# 'sarang' 파일이 없으므로 'jeongseong' 사용
name = 'jeongseong'

diet_db_path = f'../data/sarang_DB/processed_DB/DIET_{name}.xlsx'
menu_db_path = f'../data/sarang_DB/processed_DB/Menu_ingredient_nutrient_{name}.xlsx'
ingre_db_path = f'../data/sarang_DB/processed_DB/Ingredient_Price_{name}.xlsx'

diet_db, nutrient_constraints, harmony_matrix, menus, menu_counts, all_menus = load_data(diet_db_path, menu_db_path, ingre_db_path)

# Weekly_diet_ex.xlsx가 없으므로 기존 DIET 데이터 사용
initial_diet_path = diet_db_path
print("\n" + "="*60)
print("식단 최적화 알고리즘 비교 시작")
print("="*60)
print(f"데이터: {name}")
print(f"세대 수: 100")
print(f"알고리즘: NSGA-II, NSGA-III, SPEA2, ε-MOEA")
print("="*60 + "\n")

# 결과는 (all_results, performance_results) 튜플로 반환됨
optimization_results, performance_results = compare_optimizers(
        initial_diet_path,
        diet_db, menu_db_path, ingre_db_path, all_menus, nutrient_constraints, harmony_matrix, 
        generations=100,
    )